# Chapter 6. Odds and Addends
[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap06.html)

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.

## Imports

In [ ]:
import DataFrames as Dfs
import Distributions as Dsts
import CairoMakie as Cmk

In [ ]:
include("./pmf.jl")
import .ProbabilityMassFunction as pmf

### Odds

Odds [odds = $\frac{P(success)}{P(failure)} = \frac{P(success)}{1 -
P(success)}$], are used in the form of so called **Bayes's Rule**.

Bayes’s Rule is convenient if you want to do a Bayesian update on paper or in
your head. It also sheds light on the important idea of evidence and how we can
quantify the strength of evidence.

In [ ]:
function getOdds(prob::Float64)::Float64
    @assert (0 <= prob <= 1) "prob takes value in range [0-1]"
    return prob / (1 - prob)
end

In [ ]:
# if prob of winning is 75% then the odds are
# 3 to 1
getOdds(0.75)

In [ ]:
# odds in favor (1 to 9)
getOdds(0.1)

In [ ]:
# we may prefer to report odds agains 9:1
getOdds(1-0.1)

We can also do the opposite transformation, below my (BL) convesion for practice

$odds = \frac{p}{1-p}$

$\frac{p}{1-p} = odds$

$\frac{p}{1-p} = odds$ # multiply both sides by (1-p)

$p = odds * (1-p)$ # multiply: odds * (1-p)

$p = odds - odds * p$ # move (odds*p) to the left with the opposite sign

$p + (odds * p) = odds$ # rewrite left side to different equivalent form

$p * (1 + odds) = odds$ # move (1+odds) to the right with the opposite sign

$p = odds / (1 + odds)$

Below, a Julia's function for that calculation:

In [ ]:
function getProb(odds::T)::T where T<:Union{Rational, Float64}
    return odds / (odds + 1) 
end

In [ ]:
getProb(3/2),
getProb(3//2)

In [ ]:
function getProb(yes::Int, no::Int)::Float64
    return yes / (yes + no) 
end

In [ ]:
getProb(3, 2)

[...] some computations are easier when we work with odds, [...], and some
computations are even easier with log odds, [...].